In [1]:
%load_ext autoreload
%autoreload 2 #파이썬 코드를 실행하기 전에 항상 모든 모듈을 Reload

In [2]:
from posFARMlrn import posLearningScheduler
posLearningScheduler.getServerStatus()

['P0', 'P8']

In [3]:
#============ Multi-GPU ==========
from multi_gpu import to_multi_gpu
posLearningScheduler.getServerStatus()

Using TensorFlow backend.


['P0', 'P0']

In [4]:
# -*- coding: utf-8 -*-
# 딥러닝 관련 Keras 라이브러리
import keras
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.preprocessing.image import ImageDataGenerator
###### from keras.utils.training_utils import multi_gpu_model
import keras.backend.tensorflow_backend as K

# File I/O
import subprocess
import shutil
import os, sys
from glob import glob
from datetime import datetime
import argparse

# 데이터 처리
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold

# 이미지 처리
import cv2
from scipy.ndimage import rotate
import scipy.misc


/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
posLearningScheduler.getServerStatus()

['P0', 'P0']

In [6]:
def getNormpath4win32(in_path_filename):
    #in_path_filename : '../input/train/c0\\img_14779.jpg'
    #rs :  '../input/train/c0/img_14779.jpg'
    if sys.platform == 'win32':
        rs = in_path_filename.replace('\\','/')
    else:
        rs = in_path_filename
    return rs

In [7]:
# args[1] = 'd/'                       # <- feather데이터 파일의 상대 경로
# args[2] = 'datatest.feather'         # <- feather데이터 파일파일명
# args[3] = 'test.log'                 # <- log 파일명

In [8]:
#---------------------------------------------------------------------------------------------
# jupyter에서는 아래 명령이 안먹는다. (A10)
#args = parser.parse_args()  #######  터미널 모드 적용 시 uncomment해야 함

# 그래서(A10 의 사유로) 아래와 같이 변경함
class ArgumentParser1():
    pass
args = ArgumentParser1()
args.model = 'resnet50' # vgg16
args.weights, args.learning_rate, args.semi_train =  None, 1e-4, None
args.batch_size = 16 
args.random_split = 1 
args.data_augment  = 0 # data augment 1로 setting

In [9]:
args.model, args.weights, args.learning_rate, args.semi_train

('resnet50', None, 0.0001, None)

In [10]:
fc_size = 2048
n_class = 3
seed = 10
nfolds = 5
test_nfolds = 3
img_row_size, img_col_size = 224, 224

In [11]:
#train_path = r'../input/train'
train_path = r'./base_dir/train_dir'
if args.semi_train is not None:
    train_path = args.semi_train
    args.semi_train = True
# test_path = r'../input/test'
test_path = r'./base_dir/test_dir'
valid_path = r'./base_dir/valid_dir' #temp_valid_fold
labels = ['c0', 'c1', 'c2']

suffix = 'm{}.w{}.lr{}.s{}.nf{}.semi{}.b{}.row{}col{}.rsplit{}.augment{}.d{}'.format(args.model, args.weights, 
    args.learning_rate, seed, nfolds, args.semi_train, args.batch_size, img_row_size, img_col_size, args.random_split, 
    args.data_augment, datetime.now().strftime("%Y-%m-%d-%H-%M"))

# temp_train_fold 

In [12]:
# temp_train_fold = r'../input/train_{}'.format(suffix)
# temp_valid_fold = r'../input/valid_{}'.format(suffix)
temp_train_fold = train_path
temp_valid_fold = valid_path

cache = r'../cache/{}'.format(suffix)
subm = r'../subm/{}'.format(suffix)

In [13]:
def _clear_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)
for path in [cache, subm]:
    _clear_dir(path)    
# for path in [temp_train_fold, temp_valid_fold, cache, subm]:
#     _clear_dir(path)

In [14]:
posLearningScheduler.getServerStatus()

['P0', 'P0']

In [15]:
def get_model():
    #with K.tf.device('/device:CPU:0'):   #/device:CPU:0 https://3months.tistory.com/206
    # 최상위 전결층을 제외한 모델을 불러온다
    if args.weights == 'None':
        args.weights = None
    if args.model in ['vgg16']:
        base_model = keras.applications.vgg16.VGG16(include_top=False, weights=args.weights, input_shape=(img_row_size, img_col_size,3))
    elif args.model in ['vgg19']:
        base_model = keras.applications.vgg19.VGG19(include_top=False, weights=args.weights, input_shape=(img_row_size, img_col_size,3))
    elif args.model in ['resnet50']:
        base_model = keras.applications.resnet50.ResNet50(include_top=False, weights=args.weights, input_shape=(img_row_size, img_col_size,3))
    else:
        print('# {} is not a valid value for "--model"'.format(args.model))
        exit()

    # 최상위 전결층을 정의한다
    out = Flatten()(base_model.output)
    out = Dense(fc_size, activation='relu')(out)
    out = Dropout(0.5)(out)
    out = Dense(fc_size, activation='relu')(out)
    out = Dropout(0.5)(out)
    output = Dense(n_class, activation='softmax')(out)
    model = Model(inputs=base_model.input, outputs=output)

    # https://3months.tistory.com/211?category=755917
    # from keras.utils.training_utils import multi_gpu_model
    #model = multi_gpu_model(model, gpus=2)  #@@@ gpu 2개
    #============ Multi-GPU ============
    model = to_multi_gpu(model,n_gpus=1)  #n_gpus=2
    #===================================

    # SGD Optimizer를 사용하여, 모델을 compile한다
    sgd = SGD(lr=args.learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
def read_image(path):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [17]:
def crop_center(img, cropx, cropy):
    # 이미지 중간을 Crop하는 함수를 정의한다
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [18]:
def preprocess(image):
    # rescale
    image /= 255.

    # rotate
    rotate_angle = np.random.randint(40) - 20
    image = rotate(image, rotate_angle)

    # translate
    rows, cols, _ = image.shape
    width_translate = np.random.randint(60) - 30
    height_translate = np.random.randint(60) - 30
    M = np.float32([[1,0,width_translate],[0,1,height_translate]])
    image = cv2.warpAffine(image,M,(cols,rows))
    
    # zoom
    width_zoom = int(img_row_size * (0.8 + 0.2 * (1 - np.random.random())))
    height_zoom = int(img_col_size * (0.8 + 0.2 * (1 - np.random.random())))
    final_image = np.zeros((height_zoom, width_zoom, 3))
    final_image[:,:,0] = crop_center(image[:,:,0], width_zoom, height_zoom)
    final_image[:,:,1] = crop_center(image[:,:,1], width_zoom, height_zoom)
    final_image[:,:,2] = crop_center(image[:,:,2], width_zoom, height_zoom)

    # resize
    image = cv2.resize(final_image, (img_row_size, img_col_size))
    return image

#-----------------------------------------------------------------------------------------------------------------------
print('# Train Model')
# 이미지 데이터 전처리를 수행하는 함수를 정의한다
# 실시간 전처리를 추가할 경우, 전처리 함수를 설정값에 넣어준다
if args.data_augment:
    datagen = ImageDataGenerator(preprocessing_function=preprocess)
else:
    #datagen = ImageDataGenerator()
    datagen = ImageDataGenerator(rescale=1./255)

# Train Model


### preprocessing

In [19]:
print('temp_train_fold:', temp_train_fold)
print('temp_valid_fold:', temp_valid_fold)
print('test_path:', test_path)

temp_train_fold: ./base_dir/train_dir
temp_valid_fold: ./base_dir/valid_dir
test_path: ./base_dir/test_dir


In [20]:
def generate_split():
    # 이미지 생성기를 위하여 임시 훈련/검증 폴더를 생성한다
    def _generate_temp_folder(root_path):
        #os.mkdir(root_path)
        _clear_dir(root_path)
        for i in range(n_class):
            os.mkdir('{}/c{}'.format(root_path, i))
    _generate_temp_folder(temp_train_fold)
    _generate_temp_folder(temp_valid_fold)
    
    # 임시 훈련/검증 폴더에 데이터를 랜덤하게 복사한다
    train_samples = 0
    valid_samples = 0
    for label in labels:
        files = glob('{}/{}/*jpg'.format(train_path, label))
        for fl in files:
            cmd = 'cp {} {}/{}/{}'
            if np.random.randint(nfolds) != 1:
                # 데이터의 4/5를 훈련 데이터에 추가한다
                cmd = cmd.format(fl, temp_train_fold, label, os.path.basename(fl))
                train_samples += 1
            else:
                # 데이터의 1/5를 검증 데이터에 추가한다
                cmd = cmd.format(fl, temp_valid_fold, label, os.path.basename(fl))
                valid_samples += 1
            # 원본 훈련 데이터를 임시 훈련/검증 데이터에 복사한다
            subprocess.call(cmd, stderr=subprocess.STDOUT, shell=True)
    # 훈련/검증 데이터 개수를 출력한다
    print('# {} train samples | {} valid samples'.format(train_samples, valid_samples))
    return train_samples, valid_samples            

In [22]:
def generate_split99():
    df_train = pd.read_csv('./base_dir/train_dir.csv') # encoding='euc-kr'
    df_val = pd.read_csv('./base_dir/valid_dir.csv') # encoding='euc-kr'
    df_test = pd.read_csv('./base_dir/test_dir.csv') # encoding='euc-kr'    
    train_samples, valid_samples = len(df_train), len(df_val)
    return train_samples, valid_samples   

### train model

In [23]:
epochs=10

In [24]:
fold=0 # kfold를 사용하지 않는다.
# for fold in range(nfolds):
# 새로운 모델을 정의한다
model = get_model()

# 훈련/검증 데이터를 생성한다
train_samples, valid_samples = generate_split99()
# 훈련/검증 데이터 생성기를 정의한다
train_generator = datagen.flow_from_directory(
        directory=temp_train_fold,
        target_size=(img_row_size, img_col_size),
        batch_size=args.batch_size,
        class_mode='categorical',
        seed=seed)
valid_generator = datagen.flow_from_directory(
        directory=temp_valid_fold,
        target_size=(img_row_size, img_col_size),
        batch_size=args.batch_size,
        class_mode='categorical',
        seed=seed)

weight_path = '../cache/{}/weight.fold_{}.h5'.format(suffix, fold)
print('weight_path : ', weight_path)
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
        ModelCheckpoint(weight_path, monitor='val_loss', save_best_only=True, verbose=0)]

# 모델을 학습한다. val_loss 값이 3 epoch 연속 개악되면, 학습을 멈추고 최적 weight를 저장한다
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples/args.batch_size,
        epochs=epochs,
        validation_data=valid_generator,
        validation_steps=valid_samples/args.batch_size,
        shuffle=True,
        callbacks=callbacks,
        verbose=1)   

/opt/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 6732 images belonging to 3 classes.
Found 1772 images belonging to 3 classes.
weight_path :  ../cache/mresnet50.wNone.lr0.0001.s10.nf5.semiNone.b16.row224col224.rsplit1.augment0.d2020-07-21-21-28/weight.fold_0.h5
Epoch 1/10
421/420 [==============================] - 166s 394ms/step - loss: 1.5706 - acc: 0.5295 - val_loss: 0.9911 - val_acc: 0.7049
Epoch 2/10
421/420 [==============================] - 151s 358ms/step - loss: 0.8628 - acc: 0.6592 - val_loss: 0.6788 - val_acc: 0.7387
Epoch 3/10
421/420 [==============================] - 151s 360ms/step - loss: 0.6794 - acc: 0.7362 - val_loss: 0.5369 - val_acc: 0.7940
Epoch 4/10
421/420 [==============================] - 151s 359ms/step - loss: 0.5646 - acc: 0.7732 - val_loss: 0.4939 - val_acc: 0.8042
Epoch 5/10
421/420 [==============================] - 151s 358ms/step - loss: 0.5299 - acc: 0.7862 - val_loss: 0.4112 - val_acc: 0.8352
Epoch 6/10
421/420 [==============================] - 151s 358ms/step - loss: 0.5075 - acc: 0.8028 - 

In [54]:
import time
print(time.strftime('%c', time.localtime(time.time())))

Wed Jul 22 06:40:20 2020


# load_weights and continue training

In [55]:
print("다음 날 아침, 다시 학습 시작")
print(time.strftime('%c', time.localtime(time.time())))

epochs=5
weight_path = '../cache/{}/weight.fold_{}.h5'.format(suffix, fold)
print('weight_path : ', weight_path)
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
        ModelCheckpoint(weight_path, monitor='val_loss', save_best_only=True, verbose=0)]

# 모델을 학습한다. val_loss 값이 3 epoch 연속 개악되면, 학습을 멈추고 최적 weight를 저장한다
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples/args.batch_size,
        epochs=epochs,
        validation_data=valid_generator,
        validation_steps=valid_samples/args.batch_size,
        shuffle=True,
        callbacks=callbacks,
        verbose=1)   

다음 날 아침, 다시 학습 시작
Wed Jul 22 06:40:36 2020
weight_path :  ../cache/mresnet50.wNone.lr0.0001.s10.nf5.semiNone.b16.row224col224.rsplit1.augment0.d2020-07-21-21-28/weight.fold_0.h5
Epoch 1/5
421/420 [==============================] - 152s 361ms/step - loss: 0.4290 - acc: 0.8348 - val_loss: 0.3421 - val_acc: 0.8679
Epoch 2/5
421/420 [==============================] - 152s 360ms/step - loss: 0.4152 - acc: 0.8347 - val_loss: 0.3968 - val_acc: 0.8493
Epoch 3/5
421/420 [==============================] - 150s 357ms/step - loss: 0.4126 - acc: 0.8383 - val_loss: 0.3495 - val_acc: 0.8674
Epoch 4/5
421/420 [==============================] - 151s 358ms/step - loss: 0.3925 - acc: 0.8415 - val_loss: 0.3321 - val_acc: 0.8612
Epoch 5/5
421/420 [==============================] - 151s 359ms/step - loss: 0.3771 - acc: 0.8497 - val_loss: 0.3407 - val_acc: 0.8708


### test model

In [56]:
df_test.head()

,Folder,Label,File
0,./test_dir/c0,c0,1483104838__BBSFILEhealth07_1_.jpg
1,./test_dir/c0,c0,20130228_151608a.jpg
2,./test_dir/c0,c0,7508b3dbe77877b262428cf687f7e9140c09d3ee_1_.jpg
3,./test_dir/c0,c0,B9781416056638000033_f003_004_9781416056638_1_...
4,./test_dir/c0,c0,KakaoTalk_20200624_181118241_07.jpg


In [57]:
test_df = pd.read_csv('./base_dir/test_dir.csv') # encoding='euc-kr'    

In [58]:
test_path='../input/test/imgs'

In [59]:
print("다음 날 아침, 추가 학습 후 다시 test")
print(time.strftime('%c', time.localtime(time.time())))
datagen = ImageDataGenerator(rescale=1./255)
test_gen = datagen.flow_from_dataframe(
                                dataframe = test_df.sort_values(by="Label").reset_index(drop=True),
                                directory = test_path,  # img_folder + "/2ndSet_ThkConcat/"
                                x_col = "File",
                                target_size = (img_row_size, img_col_size), # input_model.input_shape[1:3],  
                                batch_size = 1,
                                class_mode = None,
                                shuffle = False)  
test_gen.reset() 
#proba = model.predict_generator(test_gen, steps=len(test_gen.filenames))
proba = model.predict_generator(test_gen, steps=len(test_gen.filenames))

pred_lab = []
for a_prob in proba :
    pred_lab.append(labels[np.argmax(a_prob)])

다음 날 아침, 추가 학습 후 다시 test
Wed Jul 22 07:01:07 2020
Found 355 images.


In [60]:
pred_lab[:5]

['c0', 'c0', 'c0', 'c0', 'c1']

In [61]:
proba_df = pd.DataFrame(proba, columns=labels)

In [62]:
pred_df = pd.DataFrame({'filename':test_gen.filenames, 'pred':pred_lab})

In [63]:
df_f = pd.merge(test_df[["File", "Label"]], pred_df, left_on="File", right_on="filename", how="left")
df_f.head()

,File,Label,filename,pred
0,1483104838__BBSFILEhealth07_1_.jpg,c0,1483104838__BBSFILEhealth07_1_.jpg,c0
1,20130228_151608a.jpg,c0,20130228_151608a.jpg,c0
2,7508b3dbe77877b262428cf687f7e9140c09d3ee_1_.jpg,c0,7508b3dbe77877b262428cf687f7e9140c09d3ee_1_.jpg,c0
3,B9781416056638000033_f003_004_9781416056638_1_...,c0,B9781416056638000033_f003_004_9781416056638_1_...,c0
4,KakaoTalk_20200624_181118241_07.jpg,c0,KakaoTalk_20200624_181118241_07.jpg,c0


In [64]:
df_f.rename(columns={"Label":"real"}, inplace=True)
df_f.head()

,File,real,filename,pred
0,1483104838__BBSFILEhealth07_1_.jpg,c0,1483104838__BBSFILEhealth07_1_.jpg,c0
1,20130228_151608a.jpg,c0,20130228_151608a.jpg,c0
2,7508b3dbe77877b262428cf687f7e9140c09d3ee_1_.jpg,c0,7508b3dbe77877b262428cf687f7e9140c09d3ee_1_.jpg,c0
3,B9781416056638000033_f003_004_9781416056638_1_...,c0,B9781416056638000033_f003_004_9781416056638_1_...,c0
4,KakaoTalk_20200624_181118241_07.jpg,c0,KakaoTalk_20200624_181118241_07.jpg,c0


In [65]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(df_f["real"], df_f["pred"])
accuracy

0.8366197183098592

In [66]:
from sklearn.metrics import classification_report 

In [67]:
target_names = ['c0', 'c1', 'c2']

In [68]:
print(classification_report(df_f["real"], df_f["pred"], target_names=target_names))

             precision    recall  f1-score   support

         c0       0.93      0.85      0.89       117
         c1       0.89      0.80      0.84       120
         c2       0.72      0.86      0.78       118

avg / total       0.85      0.84      0.84       355



In [27]:
print(len(os.listdir('train_dir/c0')))
print(len(os.listdir('train_dir/c2')))
print(len(os.listdir('train_dir/c1')))

2208
2245
2280


In [28]:
print(len(os.listdir('valid_dir/c0')))
print(len(os.listdir('valid_dir/c2')))
print(len(os.listdir('valid_dir/c1')))

581
591
600


In [29]:
print(len(os.listdir('test_dir/c0')))
print(len(os.listdir('test_dir/c2')))
print(len(os.listdir('test_dir/c1')))

117
118
120


# 아래는 사용하지 않는다